In [1]:
# yolo 모델(YOLO: Real-Time Object Detection)
# You Only Look Once, Real-Time Object Detection
# 이미지를 일정 분할로 그리드한 후 분류
# 320 × 320 : fast, low accuracy
# 609 × 609 : slow, high accuracy
# 416 × 416 : medium

import numpy as np
import cv2
import sys

def construct_yolo_v3():
    f=open('c:/data/models/coco_names.txt', 'r')
    class_names=[line.strip() for line in f.readlines()]
    model=cv2.dnn.readNet('c:/data/models/yolov3.weights','c:/data/models/yolov3.cfg')
    layer_names=model.getLayerNames()
    out_layers=[layer_names[i-1] for i in model.getUnconnectedOutLayers()]    
    return model,out_layers,class_names


def yolo_detect(img,yolo_model,out_layers):

    height,width=img.shape[0],img.shape[1]
    test_img=cv2.dnn.blobFromImage(img,1.0/256,(448,448),(0,0,0),swapRB=True)

    

    yolo_model.setInput(test_img)

    output3=yolo_model.forward(out_layers)

    

    box,conf,id=[],[],[]        # 박스, 신뢰도, 클래스

    for output in output3:

        for vec85 in output:

            scores=vec85[5:]

            class_id=np.argmax(scores)

            confidence=scores[class_id]

            if confidence>0.5:    # 신뢰도가 50% 이상만 선택

                centerx,centery=int(vec85[0]*width),int(vec85[1]*height)

                w,h=int(vec85[2]*width),int(vec85[3]*height)

                x,y=int(centerx-w/2),int(centery-h/2)

                box.append([x,y,x+w,y+h])

                conf.append(float(confidence))

                id.append(class_id)

    # 노이즈 제거 : Non maximum suppression, 최대가 아닌 박스들을 제거하는 기법

    #              비  최대       억제

    ind=cv2.dnn.NMSBoxes(box,conf,0.5,0.4)

    objects=[box[i]+[conf[i]]+[id[i]] for i in range(len(box)) if i in ind]

    return objects

# YOLO 모델 생성

model,out_layers,class_names=construct_yolo_v3()

colors=np.random.uniform(0,255,size=(len(class_names),3))

img=cv2.imread('c:/data/images/soccer.jpg')

if img is None: sys.exit('파일이 없습니다.')

# 객체 인식

res=yolo_detect(img,model,out_layers)

for i in range(len(res)):

    x1,y1,x2,y2,confidence,id=res[i]

    text=str(class_names[id])+'%.3f'%confidence

    cv2.rectangle(img,(x1,y1),(x2,y2),colors[id],2)

    cv2.putText(img,text,(x1,y1+30),cv2.FONT_HERSHEY_PLAIN,1.5,colors[id],2)

cv2.imshow("Object detection by YOLO v.3",img)

cv2.waitKey()

cv2.destroyAllWindows()

#마지막으로 모든 정보를 추출하여 화면에 표시합니다.

#Box : 감지된 개체를 둘러싼 사각형의 좌표

#Label : 감지된 물체의 이름

#Confidence : 0에서 1까지의 탐지에 대한 신뢰도